In [39]:
# import dependencies
import cv2
import mediapipe as mp

In [40]:
# initialize the mediapipe poses and drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [57]:
# setup mediapipe
cap = cv2.VideoCapture(0)

# check if camera can open
if not cap.isOpened():
    print("Could not open camera")
    exit()
    
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # invert the video
        frame = cv2.flip(frame, 1)
        
        # recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False       # makes image read-only for detection
        
        # make the detection
        results = pose.process(image)
        
        # recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # get landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            # print(landmarks)
            print(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value])
        except AttributeError:
            pass
        
        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(0, 0, 255)), mp_drawing.DrawingSpec(color=(0, 255, 0)))
        # print(image)
        
        cv2.imshow('Backend Limb Detection', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
                
cv2.destroyAllWindows()
cap.release()

x: 0.740536332
y: 0.708557069
z: -0.229810387
visibility: 0.9998

x: 0.740727544
y: 0.705351174
z: -0.216706246
visibility: 0.999794781

x: 0.741673589
y: 0.705104947
z: -0.202623293
visibility: 0.999776602

x: 0.743445456
y: 0.70511812
z: -0.198042303
visibility: 0.999731362

x: 0.746154785
y: 0.705979228
z: -0.171528697
visibility: 0.999671698

x: 0.747198939
y: 0.7080428
z: -0.165015548
visibility: 0.999586463

x: 0.747555196
y: 0.709391296
z: -0.148779139
visibility: 0.999472201

x: 0.748242199
y: 0.719802201
z: -0.119171381
visibility: 0.999322295

x: 0.748389661
y: 0.723118246
z: -0.103883006
visibility: 0.999238551

x: 0.749378622
y: 0.730321348
z: -0.100564465
visibility: 0.999180913

x: 0.744478106
y: 0.723430574
z: -0.100810096
visibility: 0.998966396

x: 0.741375625
y: 0.716487646
z: -0.111395739
visibility: 0.998805642

x: 0.739121318
y: 0.711346388
z: -0.117154658
visibility: 0.998716712

x: 0.734806955
y: 0.701757967
z: -0.119930968
visibility: 0.998734653

x: 0.732870281

In [48]:
len(landmarks)

33

In [49]:
for lndmk in mp_pose.PoseLandmark:
    print(lndmk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [50]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.763607442
y: 0.678792238
z: -1.21098971
visibility: 0.99309653